In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train = train.rename(columns={'賃料':'target', '契約期間':'Contract', '間取り':'Room', 
                              '築年数':'Passed', '駐車場':'Parking', '室内設備':'Facility', 
                              '放送・通信':'Internet', '周辺環境':'Building', '建物構造':'Material', 
                              '面積':'Area', 'キッチン':'Kitchen', '所在地':'Place',
                              'バス・トイレ':'Bath', '所在階':'Floor', 'アクセス':'Access', 
                              '方角':'Angle'})


test = test.rename(columns={'契約期間':'Contract', '間取り':'Room', '築年数':'Passed', 
                            '駐車場':'Parking', '室内設備':'Facility', '放送・通信':'Internet', 
                            '周辺環境':'Building', '建物構造':'Material', '面積':'Area', 
                            'キッチン':'Kitchen', '所在地':'Place', 'バス・トイレ':'Bath', 
                            '所在階':'Floor', 'アクセス':'Access', '方角':'Angle'})

In [3]:
data = pd.concat([train, test], sort=False)

In [4]:
data.head(2)

,id,target,Place,Access,Room,Passed,Angle,Area,Floor,Bath,Kitchen,Internet,Facility,Parking,Building,Material,Contract
0,1,75000.0,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間
1,2,76000.0,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間


In [5]:
#入力ミスの修正
data['Floor'][(data['id']==40675)] = "1"
data['target'][(data['id']==5776)] = 120350
data['Area'][(data['id']==41881)] = '10m2'
data['Area'][(data['id']==7492)] = '50.83m2'
data['Passed'][(data['id']==20428)] = '19年7ヶ月'
data['Bath'] = data['Bath'].fillna("0")
data['Kitchen'] = data['Kitchen'].fillna("0")
data['Internet'] = data['Internet'].fillna("0")
data['Facility'] = data['Internet'].fillna("0")

<ipython-input-5-774935d01e30>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Floor'][(data['id']==40675)] = "1"
<ipython-input-5-774935d01e30>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'][(data['id']==5776)] = 120350
<ipython-input-5-774935d01e30>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Area'][(data['id']==41881)] = '10m2'
<ipython-input-5-774935d01e30>:5: SettingWithCopyWarning: 
A value is t

In [6]:
#23区の名前を取得
def get_ward(x):
    x = x[x.find('都')+1:x.find('区')]
    return x
data['ward'] = data['Place'].apply(get_ward)

In [7]:
def land_price(x):
    if x  == '北':
        x = 70
    elif x  == '中央':
        x = 552
    elif x  == '渋谷':
        x = 403
    elif x  == '杉並':
        x = 71
    elif x  == '葛飾':
        x = 37
    elif x  == '荒川':
        x = 68
    elif x  == '練馬':
        x = 45
    elif x  == '目黒':
        x = 118
    elif x  == '文京':
        x = 131
    elif x  == '板橋':
        x = 55
    elif x  == '大田':
        x = 57
    elif x  == '江戸川':
        x = 44
    elif x  == '港':
        x = 436
    elif x  == '墨田':
        x = 60
    elif x  == '新宿':
        x = 368
    elif x  == '中野':
        x = 91
    elif x  == '豊島':
        x = 173
    elif x  == '台東':
        x = 153
    elif x  == '足立':
        x = 42
    elif x  == '世田谷':
        x = 72
    elif x  == '江東':
        x = 63
    elif x  == '品川':
        x = 99
    elif x  == '千代田':
        x = 505
    else:
        x=0
    return x
data['land_price']= data['ward'].apply(land_price)

In [8]:
#各区の賃貸料に対する統計量をtrainデータをもとに作成①
train_static = data[:len(train)] 
ward_target_df = train_static[['target','ward']]
#最小値
min_df = ward_target_df.groupby('ward').min()
min_df['target'] = min_df['target']

#中央値
median_df = ward_target_df.groupby('ward').median()
median_df['target'] = median_df['target']

In [9]:
#①の統計量をtest,trainデータに結合②
def ward_target_min(x):
    if x  == '北':
        x = min_df.loc['北','target']
    elif x  == '中央':
        x = min_df.loc['中央','target']
    elif x  == '渋谷':
        x = min_df.loc['渋谷','target']
    elif x  == '杉並':
        x = min_df.loc['杉並','target']
    elif x  == '葛飾':
        x = min_df.loc['葛飾','target']
    elif x  == '荒川':
        x = min_df.loc['荒川','target']
    elif x  == '練馬':
        x = min_df.loc['練馬','target']
    elif x  == '目黒':
        x = min_df.loc['目黒','target']
    elif x  == '文京':
        x = min_df.loc['文京','target']
    elif x  == '板橋':
        x = min_df.loc['板橋','target']
    elif x  == '大田':
        x = min_df.loc['大田','target']
    elif x  == '江戸川':
        x = min_df.loc['江戸川','target']
    elif x  == '港':
        x = min_df.loc['港','target']
    elif x  == '墨田':
        x = min_df.loc['墨田','target']
    elif x  == '新宿':
        x = min_df.loc['新宿','target']
    elif x  == '中野':
        x = min_df.loc['中野','target']
    elif x  == '豊島':
        x = min_df.loc['豊島','target']
    elif x  == '台東':
        x = min_df.loc['台東','target']
    elif x  == '足立':
        x = min_df.loc['足立','target']
    elif x  == '世田谷':
        x = min_df.loc['世田谷','target']
    elif x  == '江東':
        x = min_df.loc['江東','target']
    elif x  == '品川':
        x = min_df.loc['品川','target']
    elif x  == '千代田':
        x = min_df.loc['千代田','target']
    else:
        x=0
    return x
data['ward_min']= data['ward'].apply(ward_target_min)

In [10]:
#①の統計量をtest,trainデータに結合②
def ward_target_median(x):
    if x  == '北':
        x = median_df.loc['北','target']
    elif x  == '中央':
        x = median_df.loc['中央','target']
    elif x  == '渋谷':
        x = median_df.loc['渋谷','target']
    elif x  == '杉並':
        x = median_df.loc['杉並','target']
    elif x  == '葛飾':
        x = median_df.loc['葛飾','target']
    elif x  == '荒川':
        x = median_df.loc['荒川','target']
    elif x  == '練馬':
        x = median_df.loc['練馬','target']
    elif x  == '目黒':
        x = median_df.loc['目黒','target']
    elif x  == '文京':
        x = median_df.loc['文京','target']
    elif x  == '板橋':
        x = median_df.loc['板橋','target']
    elif x  == '大田':
        x = median_df.loc['大田','target']
    elif x  == '江戸川':
        x = median_df.loc['江戸川','target']
    elif x  == '港':
        x = median_df.loc['港','target']
    elif x  == '墨田':
        x = median_df.loc['墨田','target']
    elif x  == '新宿':
        x = median_df.loc['新宿','target']
    elif x  == '中野':
        x = median_df.loc['中野','target']
    elif x  == '豊島':
        x = median_df.loc['豊島','target']
    elif x  == '台東':
        x = median_df.loc['台東','target']
    elif x  == '足立':
        x = median_df.loc['足立','target']
    elif x  == '世田谷':
        x = median_df.loc['世田谷','target']
    elif x  == '江東':
        x = median_df.loc['江東','target']
    elif x  == '品川':
        x = median_df.loc['品川','target']
    elif x  == '千代田':
        x = median_df.loc['千代田','target']
    else:
        x=0
    return x
data['ward_median']= data['ward'].apply(ward_target_median)

In [11]:
#各区でLabelencodhing
le = LabelEncoder()
data['ward'] = le.fit_transform(data['ward'])

In [12]:
#最寄り駅までの距離を取得
import re
def get_min_time(x):
    time_list = re.findall(r'徒歩(\d+)分',str(x))

    time_list = map(int, time_list)
 
    min_value = min(time_list)
    
    return min_value

data['Access'] = data['Access'].apply(get_min_time)

In [13]:
#部屋の面積を取得
def get_area(x):
    x = x[:-2]
    x = float(x)
    return x

data['Area'] = data['Area'].apply(get_area)

In [14]:
#方角の取得
def get_angle(x):
    if x =='南':
        x=1
    elif x=='東':
        x=2
    elif x=='西':
        x=3
    elif x=='北':
        x=4
    elif x=='南東':
        x=5
    elif x=='南西':
        x=6
    elif x=='北西':
        x=7
    else:
        x=8
    return x
data['Angle'] = data['Angle'].apply(get_angle)

In [15]:
# 部屋の数の特徴量作成

#LDK,DK,LK,R,Kの前についている数値を取得する関数
def room_size(x):
    x.split('LDK')
    x.split('DK')
    x.split('LK')
    x.split('R')
    x.split('K')
    
    return int(x[0])

def get_room_num(room):
    s_num=0
    total_room_num=0
    if ('S(納戸)' in room):
        s_num+=1
        if ('LDK' in room):
            total_room_num = room_size(room)
            total_room_num = total_room_num + 1 + s_num
        else:
            total_room_num = room_size(room) + s_num
    else:
        if ('LDK' in room):
            total_room_num = room_size(room)
            total_room_num +=1
        else:
            total_room_num = room_size(room)
    return total_room_num
data['Room'] = data['Room'].apply(get_room_num)

In [16]:
#築何年かを取得（新築は０年とする）
def get_age(x):
    if x == '新築':
        return 0
    else:
        return int(x.split('年')[0])
        
data['Passed'] = data['Passed'].apply(get_age)

In [17]:
# 所在階と最高階の情報の取得
floors = []
total_floors = []
heigth_rate = []
for i in data['Floor']:
    if ('／' in i):
        i = i.split('／')
        floor = i[0]
        total_floor = i[1]
        #所在階の取得
        if ('地下' in floor):
            underground = floor
            underground = underground.strip('地下')
            underground = underground.strip('階')
            floors.append(int(underground))
        else:
            if len(floor)==0:
                floor = 0
                floors.append(int(floor))
            else:
                floor = floor.strip('階') 
                floors.append(int(floor))
        #階数の取得 
        if ('地下' in total_floor):
            total_floor = total_floor.split('階建')
            total_floor = total_floor[0]
            total_floors.append(int(total_floor))
        else:
            total_floor = total_floor.strip('階建')
            total_floors.append(int(total_floor))
            
    else:
        floor = i[0]
        floors.append(int(floor))
        
        total_floor = 1
        total_floors.append(int(total_floor))
        
# 高さ率の取得    
floors_array = np.array(floors)
total_floors_array = np.array(total_floors)
heigth_rate = floors_array/total_floors_array

data['Floor'] = floors
data['total_floor'] = total_floors
# data['height_rate'] = heigth_rate

In [18]:
#一部屋あたりの面積の取得
data['area_mean'] = data['Area']/data['Room']
data['area_mean'] = data['area_mean']

In [19]:
#キッチン関連のコンテンツの特徴量
kitchen = []
for i in data['Kitchen']:
    i = i.replace('\t','')
    i = i.split('／')
    kitchen_contents = 0
    kitchen_contents = len(i)
    kitchen.append(kitchen_contents)
data['Kitchen'] = kitchen

In [20]:
#インターネット関連のコンテンツの特徴量
internet = []
for i in data['Internet']:
    i = i.replace('\t','')
    i = i.split('／')
    internet_contents = 0
    internet_contents = len(i)
    internet.append(internet_contents)
data['Internet'] = internet

In [21]:
#建築構造のデータをLabelencodhing
data['Material'] = le.fit_transform(data['Material'])

In [22]:
#使わない説明変数の削除
delete_columns = ['id','Contract','Parking','Place','Facility','Building','Bath']
data.drop(delete_columns, axis=1, inplace=True)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62732 entries, 0 to 31261
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   target       31470 non-null  float64
 1   Access       62732 non-null  int64  
 2   Room         62732 non-null  int64  
 3   Passed       62732 non-null  int64  
 4   Angle        62732 non-null  int64  
 5   Area         62732 non-null  float64
 6   Floor        62732 non-null  int64  
 7   Kitchen      62732 non-null  int64  
 8   Internet     62732 non-null  int64  
 9   Material     62732 non-null  int64  
 10  ward         62732 non-null  int64  
 11  land_price   62732 non-null  int64  
 12  ward_min     62732 non-null  float64
 13  ward_median  62732 non-null  float64
 14  total_floor  62732 non-null  int64  
 15  area_mean    62732 non-null  float64
dtypes: float64(5), int64(11)
memory usage: 8.1 MB


In [24]:
data.head(2)

,target,Access,Room,Passed,Angle,Area,Floor,Kitchen,Internet,Material,ward,land_price,ward_min,ward_median,total_floor,area_mean
0,75000.0,4,1,9,5,20.01,1,4,4,3,3,70,31000.0,89000.0,12,20.01
1,76000.0,5,1,44,8,16.50,5,3,1,10,1,552,40000.0,178000.0,10,16.50
